In [ ]:
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_row=10
pd.options.display.float_format='{:.1f}'.format

In [ ]:
california_housing_dataframe=pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv",sep=",")

In [ ]:
california_housing_dataframe=california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

In [ ]:
california_housing_dataframe.describe()

In [ ]:
my_feature=califronia_housing_dataframe[["total_rooms"]]
feature_columns=[tf.feature_column.numeric_column("total_rooms")]

In [ ]:
targets=california_housing_dataframe["median_house_value"]

my_optimizer=tf.train.GrandientDescentOptimizer(learning_rate=0.00001)
my_optimizer=tf.contrib.estimator.clip_grandients_by_norm(my_optimizer,5.0)

linear_regressor=tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

In [ ]:
def my_input_fn(features,targets,batch_size=1,shuffle=True,num_epochs=None):
    features={key:np.array(value) for key,value in dict(features).items()}
    ds=Dataset.from_tensor_slices((features,targets))
    ds=ds.batch(batch_size).repeat(num_epochs)
    if shuffle:
        ds=ds.shuffle(buffer_size=10000)
    features,labels=ds.make_one_shot_iterator().get_next()
    return features,labels

In [ ]:
_=linear_regressor.train(
    input_fn=lambda:my_input_fn(my_feature,targets),
    steps=100
)

In [ ]:
prediction_input_fn=lambda:my_input_fn(my_feature,targets,num_epochs=1,shuffle=False)
predictions=linear_regressor.predict(input_fn=prediction_input_fn)
predictions=np.array([item['predictions'][0] for item in predictions])
mean_squared_error=metrics.mean_squared_error(predictions,targets)
root_mean_squared_error=math.sqrt(mean_squared_error)
print "Mean Squared Error (on training data):%0.3f" %mean_squared_error
print "Root Mean Squared Error (on training data):%0.3f" %root_mean_squared_error

In [5]:
min_house_value=california_housing_dataframe["median_house_value"].min()
max_house_value=california_housing_dataframe["median_house_value"].max()
min_max_difference=max_house_value-min_house_value
print "Min Median House Value:%0.3f"%min_house_value
print "Max Median House Value:%0.3f"%max_house_value
print "Difference between Min and Max:%0.3f"%min_max_difference
print "Root Mean Squaerd Error:%0.3f"%root_mean_squared_error

SyntaxError: invalid syntax (<ipython-input-5-591cbd3b9a7b>, line 2)

In [ ]:
calibration_data=pd.DataFrame()
calibration_data["predictions"]=pd.Series(predictions)
calibration_data["target"]=pd.Series(targets)
calibration_data.describe()

In [ ]:
sample=aclifornia_housing_dataframe.sample(n=300)

In [ ]:
x_0=sample["total_rooms"].min()
x_1=sample["total_rooms"].max()
weight=linear_regressor.get_variable_value('linear/linear_model/total_rooms/weights')[0]
bias=linear_regressor.get_variable_value('linear/linear_model/bias_weights')
y_0=weight*x_0+bias
y_1=weight*x_1+bias
plt.plot([x_0,x_1],[y_0,y_1],c='r')
plt.ylable("median_house_value")
plt.xlable("total_rooms")
plt.scatter(sample["total_rooms"],sample["median_house_value"])
plt.show()

In [ ]:
def train_model(learning_rate,steps,batch_size,input_feature="total_rooms"):
    periods=10
    steps_per_perriod = steps/periods
    my_feature = input_feature
    my_feature_data=california_housing_dataframe[[my_feature]]
    my_label="median_house_value"
    targets=california_housing_dataframe[my_label]
    
    feature_columns=[tf.feature_column.numeric_column(my_feature)]
    
    training_input_fn=lambda:my_input_fn(my_feature_data,targets,batch_size=batch_size)
    predicting_input_fn=lambda:my_input_fn(my_feature_data,targets,num_epochs=1,shuffle=False)
    
    my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer=tf.contrib.estimator.clip_gradients_by_norm(my_optimizer,5.0)
    linear_regressor=tf.estimator.LinearRegressor(
        feature_columns=feature_columns,
        optimizer=my_optimizer
    )
    
    plt.figure(figsize(15,6))
    plt.subplot(1,2,1)
    plt.title("Learned Line by Period")
    plt.ylabel(my_label)
    plt.xlabel(my_feature)
    sample=california_housing_dataframe.sample(n=300)
    plt.scatter(sample[my_feathre],sample[my_label])
    colors=[cm.coolwarm(x) for x in np.linespace(-1,1,periods)]
    
    print "Training model..."
    print "RMSE (on training data):"
    root_mean_squared_errors=[]
    for period in range(0,periods):
        linear_regressor.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
    predictions=linear_regressor.predict(input_fn=prediction_input_fn)
    predictions=np.array([item['predictions'][0] for item in predictions])
    
    root_mean_squared_error=math.sqrt(metrics.mean_squared_error(predictions,targets))
    
    print "period %02d:%0.2f"%(period,root_mean_squared_error)
    
    root_mean_squared_errors.append(root_mean_squared_error)
    
    y_extents=np.array([0,sample[my_label].max()])
    weight=linear_regressor.get_variable_value('linear/linear_model/%s/weights'%input_feature)
    bias = linear_regressor.get_vairable_value('linear/linear_model/bias_weights')
    
    x_extents=(y_extents-bias)/weight
    x_extents=np.maximum(np.minimum(x_extents,
                                   sample[my_feature].max()),
                        sample[my_feature].min())
    y_extents=weight*x_extents+bias
    plt.plot(x_extents,y_extents,color=color[period])
print "Model training finished."

plt.subplot(1,2,2)
plt.ylabel('RMSE')
plt.xlabel('Periods')
plt.title("Root Mean Squared Error vs. Periods")
plt.tight_layout()
plt.plot(root_mean_squared_errors)

calibration_data=pd.DataFrame()
calibration_data["predictions"]=pd.Series(predictions)
calibration_data["targets"]=pd.Series(targets)
display.display(calibration_data.describe())
print "Final RMSE (on training data): %0.2f" % root_mean_squared_error